In [1]:
import findspark
import os
import configparser
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.sql import DataFrame
import pymongo

In [2]:

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'



dataset1_path = './Dataset/datasets_894_813759_2015.csv'
dataset2_path = './Dataset/datasets_894_813759_2016.csv'
dataset3_path = './Dataset/datasets_894_813759_2017.csv'
dataset4_path = './Dataset/datasets_894_813759_2018.csv'
dataset5_path = './Dataset/sample.txt'



print(dataset1_path)

./Dataset/datasets_894_813759_2015.csv


In [3]:
sc = SparkSession.builder.config('spark.jars.packages',
                                    'org.apache.hadoop:hadoop-aws:2.6.4').getOrCreate()

# sc._jsc.hadoopConfiguration().set("fs.s3a.access.key",
#                                      os.environ['AWS_ACCESS_KEY_ID'])
# sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key",
#                                      os.environ['AWS_SECRET_ACCESS_KEY'])
# sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

In [3]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("local")
         .setAppName("My app")
         .set("spark.executor.memory", "1g"))
sc = SparkContext(conf = conf)

In [4]:
# dataset_1_data = sc.read.csv(dataset1_path, header=True)
# dataset_2_data = sc.read.csv(dataset2_path, header=True)
# dataset_3_data = sc.read.csv(dataset3_path, header=True)
# dataset_4_data = sc.read.csv(dataset4_path, header=True)
data = sc.textFile(dataset5_path)
# dataset_5_data = sc.read.csv(dataset5_path, header=True)




In [5]:
words = data.flatMap(lambda line: line.split(" "))
wordscount = words.map(lambda word : (word, 1)).reduceByKey(lambda a, b : a + b)

In [8]:
sc

<SparkContext master=local appName=My app>

In [35]:
dataset_1_data = dataset_1_data.withColumn("Year", lit("2015"))

In [14]:
dataset_2_data = dataset_2_data.withColumn("Year", lit("2016"))
dataset_3_data = dataset_3_data.withColumn("Year", lit("2017"))
dataset_4_data = dataset_4_data.withColumn("Year", lit("2018"))
# dataset_5_data = dataset_5_data.withColumn("Year", lit("2019"))

In [36]:

dataset_1_data.toPandas()

,Country,Region,Happiness_Rank,Happiness_Score,Standard_Error,Economy (GDP per Capita),Family,Health(Life_Expectancy),Freedom,Trust(Government_Corruption),Generosity,Dystopia_Residual,Year
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.4363,2.70201,2015
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015
3,Norway,Western Europe,4,7.522,0.0388,1.459,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015
5,Finland,Western Europe,6,7.406,0.0314,1.29025,1.31826,0.88911,0.64169,0.41372,0.23351,2.61955,2015
6,Netherlands,Western Europe,7,7.378,0.02799,1.32944,1.28017,0.89284,0.61576,0.31814,0.4761,2.4657,2015
7,Sweden,Western Europe,8,7.364,0.03157,1.33171,1.28907,0.91087,0.6598,0.43844,0.36262,2.37119,2015
8,New Zealand,Australia and New Zealand,9,7.286,0.03371,1.25018,1.31967,0.90837,0.63938,0.42922,0.47501,2.26425,2015
9,Australia,Australia and New Zealand,10,7.284,0.04083,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646,2015


In [37]:
for col in list(dataset_1_data.columns):
    print(col, dataset_1_data.filter((dataset_1_data[col] == "") | dataset_1_data[col].isNull()).count())

Country 0
Region 0
Happiness_Rank 0
Happiness_Score 0
Standard_Error 0
Economy (GDP per Capita) 0
Family 0
Health(Life_Expectancy) 0
Freedom 0
Trust(Government_Corruption) 0
Generosity 0
Dystopia_Residual 0
Year 0


In [15]:
for col in list(dataset_2_data.columns):
    print(col, dataset_2_data.filter((dataset_2_data[col] == "") | dataset_2_data[col].isNull()).count())

Country 0
Region 0
Happiness Rank 0
Happiness Score 0
Lower Confidence Interval 0
Upper Confidence Interval 0
Economy (GDP per Capita) 0
Family 0
Health (Life Expectancy) 0
Freedom 0
Trust (Government Corruption) 0
Generosity 0
Dystopia Residual 0
Year 0


In [38]:
dataset_1_data.createOrReplaceTempView('dataset_1_data')

In [52]:
happiness_2015 = sc.sql(""" SELECT Country, Region, Happiness_Score
                                FROM dataset_1_data 
                                WHERE Happiness_Score > 6
                                
                                ORDER BY Happiness_Score DESC""")
happiness_2015.limit(10).toPandas()

,Country,Region,Happiness_Score
0,Switzerland,Western Europe,7.587
1,Iceland,Western Europe,7.561
2,Denmark,Western Europe,7.527
3,Norway,Western Europe,7.522
4,Canada,North America,7.427
5,Finland,Western Europe,7.406
6,Netherlands,Western Europe,7.378
7,Sweden,Western Europe,7.364
8,New Zealand,Australia and New Zealand,7.286
9,Australia,Australia and New Zealand,7.284


In [57]:
happiness_2015 = sc.sql(""" SELECT Country, Region, Happiness_Score
                                FROM dataset_1_data 
                                ORDER BY Happiness_Score""")
happiness_2015.limit(10).toPandas()

,Country,Region,Happiness_Score
0,Togo,Sub-Saharan Africa,2.839
1,Burundi,Sub-Saharan Africa,2.905
2,Syria,Middle East and Northern Africa,3.006
3,Benin,Sub-Saharan Africa,3.34
4,Rwanda,Sub-Saharan Africa,3.465
5,Afghanistan,Southern Asia,3.575
6,Burkina Faso,Sub-Saharan Africa,3.587
7,Ivory Coast,Sub-Saharan Africa,3.655
8,Guinea,Sub-Saharan Africa,3.656
9,Chad,Sub-Saharan Africa,3.667


In [45]:
happiness_2015.write.format('com.databricks.spark.csv').save('./my.csv',header = 'true')

In [63]:
sc.stop()

In [59]:
#MongoDB tutorial

In [98]:
from pymongo import MongoClient
client = pymongo.MongoClient("mongodb+srv://dbUser:<pass>@cluster0.t72w2.mongodb.net/myfirstdatabase?retryWrites=true&w=majority")
db = client.test


Collection(Database(MongoClient(host=['cluster0-shard-00-01.t72w2.mongodb.net:27017', 'cluster0-shard-00-00.t72w2.mongodb.net:27017', 'cluster0-shard-00-02.t72w2.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-o9qwvt-shard-0', ssl=True), 'test'), 'test-collection')

In [99]:
mydb = client["firstdatabase"]
mycol = mydb["customers"]

mydict = { "name": "John", "address": "Highway 37" }



In [100]:
mydict = { "name": "John", "address": "Highway 37" }

In [101]:
x = mycol.insert_one(mydict)

In [102]:
post_1 = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun, you guys',
    'author': 'Scott'
}
post_2 = {
    'title': 'Virtual Environments',
    'content': 'Use virtual environments, you guys',
    'author': 'Scott'
}
post_3 = {
    'title': 'Learning Python',
    'content': 'Learn Python, it is easy',
    'author': 'Bill'
}
new_result = mycol.insert_many([post_1, post_2, post_3])
print('Multiple posts: {0}'.format(new_result.inserted_ids))

Multiple posts: [ObjectId('5f2bcdfdd8def81850e9e781'), ObjectId('5f2bcdfdd8def81850e9e782'), ObjectId('5f2bcdfdd8def81850e9e783')]


In [95]:
bills_post = mycol.find_one({'author': 'Bill'})
print(bills_post)

{'_id': ObjectId('5f2bcca4d8def81850e9e77d'), 'title': 'Learning Python', 'content': 'Learn Python, it is easy', 'author': 'Bill'}


In [ ]:
mycol = mydb["customers"]

In [39]:
sc.stop()